In [103]:
import pandas as pd
import kagglehub
import shutil
import os
import ast

In [104]:
# Download the dataset
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

# Define current directory
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "data")

# Copy all files from the kagglehub cache folder to current dir
for filename in os.listdir(path):
    src = os.path.join(path, filename)
    dst = os.path.join(data_dir, filename)
    if os.path.isfile(src):
        shutil.copy(src, dst)

print(f"All files copied to: {data_dir}")
os.rename("data/keywords.csv", "data/movies_keywords.csv")


All files copied to: /Users/vikram/Desktop/Year 4/Semester 2/Machine Learning/Project/cs451-project/data


In [105]:
# Function to extract genre names
def extract_comma_separated(df, column_name, key='name'):
    """
    Converts stringified lists of dicts in a column into comma-separated values of a given key.
    
    Args:
        df (pd.DataFrame): Your DataFrame
        column_name (str): The name of the column to process
        key (str): The key to extract from each dictionary in the list
    
    Returns:
        pd.Series: A cleaned column with comma-separated strings
    """
    def extract_values(obj_str):
        try:
            items = ast.literal_eval(obj_str)
            return ', '.join([str(item[key]) for item in items if key in item])
        except (ValueError, SyntaxError, TypeError):
            return None

    return df[column_name].apply(extract_values)

def extract_single_dict_value(obj_str, key='name'):
    """
    Extracts the value of a given key from a stringified dictionary.

    Args:
        obj_str (str): The stringified dictionary
        key (str): The key to extract

    Returns:
        str or None: The value corresponding to the key, or None if invalid
    """
    try:
        obj = ast.literal_eval(obj_str)
        return str(obj.get(key))
    except (ValueError, SyntaxError, AttributeError):
        return None
    
# Load your dataset
df = pd.read_csv("data/movies_metadata.csv", low_memory=False)
metadata = df.copy()
print(f"Inital Count: {metadata.shape[0]}")
metadata = metadata[metadata['status'] == 'Released']
print(f"Released Count: {metadata.shape[0]}")
metadata = metadata[metadata['vote_count'] >= 100]
print(f"Sufficient Votes Count: {metadata.shape[0]}")

# Apply the function to the 'genres' column
metadata['collection_name'] = df['belongs_to_collection'].apply(lambda x: extract_single_dict_value(x, 'name'))
metadata['collection_id'] = df['belongs_to_collection'].apply(lambda x: extract_single_dict_value(x, 'id'))
metadata['genres'] = extract_comma_separated(df, 'genres', key='name')
metadata['genre_ids'] = extract_comma_separated(df, 'genres', key='id')
metadata['production_companies'] = extract_comma_separated(df, 'production_companies', key='name')
metadata['production_companies_ids'] = extract_comma_separated(df, 'production_companies', key='id')
metadata['production_countries'] = extract_comma_separated(df, 'production_countries', key='iso_3166_1')
metadata['spoken_languages'] = extract_comma_separated(df, 'spoken_languages', key='name')
metadata.drop(columns=['belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path', 'status', 'tagline', 'video'], inplace=True)

# Preview result
display(metadata.head())

Inital Count: 45466
Released Count: 45014
Sufficient Votes Count: 6052


,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,collection_name,collection_id,genre_ids,production_companies_ids
0,False,30000000,"Animation, Comedy, Family",862,en,21.946943,Pixar Animation Studios,US,1995-10-30,373554033.0,81.0,English,Toy Story,7.7,5415.0,Toy Story Collection,10194,"16, 35, 10751",3
1,False,65000000,"Adventure, Fantasy, Family",8844,en,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",US,1995-12-15,262797249.0,104.0,"English, Français",Jumanji,6.9,2413.0,None,None,"12, 14, 10751","559, 2550, 10201"
4,False,0,Comedy,11862,en,8.387519,"Sandollar Productions, Touchstone Pictures",US,1995-02-10,76578911.0,106.0,English,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,96871,35,"5842, 9195"
5,False,60000000,"Action, Crime, Drama, Thriller",949,en,17.924927,"Regency Enterprises, Forward Pass, Warner Bros.",US,1995-12-15,187436818.0,170.0,"English, Español",Heat,7.7,1886.0,None,None,"28, 80, 18, 53","508, 675, 6194"
6,False,58000000,"Comedy, Romance",11860,en,6.677277,"Paramount Pictures, Scott Rudin Productions, M...","DE, US",1995-12-15,0.0,127.0,"Français, English",Sabrina,6.2,141.0,None,None,"35, 10749","4, 258, 932, 5842, 14941, 55873, 58079"


In [106]:
# Load the DataFrame
df = pd.read_csv("data/credits.csv")

# Define a function to simplify and convert types
def simplify_people(credits_str):
    try:
        return ast.literal_eval(credits_str)
    except (ValueError, SyntaxError):
        return []

# Apply to both 'cast' and 'crew'
credits = pd.DataFrame()
credits['id'] = df['id'].astype(str)
credits['cast'] = df['cast'].apply(simplify_people)
credits['crew'] = df['crew'].apply(simplify_people)

# Extract the top 5 cast members
for i in range(5):
    credits[f'actor_{i+1}_name'] = credits['cast'].apply(
        lambda x: x[i]['name'] if len(x) > i else None
    )
    credits[f'actor_{i+1}_gender'] = credits['cast'].apply(
        lambda x: str(x[i]['gender']) if len(x) > i else None
    )
    credits[f'actor_{i+1}_id'] = credits['cast'].apply(
        lambda x: str(x[i]['id']) if len(x) > i else None
    )

# Updated function to extract director info
def extract_director_info(crew_list):
    for person in crew_list:
        if person.get('job') == 'Director':
            return {
                'name': person.get('name'),
                'gender': str(person.get('gender')),
                'id': str(person.get('id'))
            }
    return {'name': None, 'gender': None, 'id': None}

# Apply the function
credits['director_info'] = credits['crew'].apply(extract_director_info)

# Split into separate columns
credits['director_name'] = credits['director_info'].apply(lambda x: x['name'])
credits['director_gender'] = credits['director_info'].apply(lambda x: x['gender'])
credits['director_id'] = credits['director_info'].apply(lambda x: x['id'])

# Drop the original 'cast', 'crew', and 'director_info' columns
credits = credits.drop(columns=['cast', 'crew', 'director_info'])

credits.head()


,id,actor_1_name,actor_1_gender,actor_1_id,actor_2_name,actor_2_gender,actor_2_id,actor_3_name,actor_3_gender,actor_3_id,actor_4_name,actor_4_gender,actor_4_id,actor_5_name,actor_5_gender,actor_5_id,director_name,director_gender,director_id
0,862,Tom Hanks,2,31,Tim Allen,2,12898,Don Rickles,2,7167,Jim Varney,2,12899,Wallace Shawn,2,12900,John Lasseter,2,7879
1,8844,Robin Williams,2,2157,Jonathan Hyde,2,8537,Kirsten Dunst,1,205,Bradley Pierce,0,145151,Bonnie Hunt,1,5149,Joe Johnston,2,4945
2,15602,Walter Matthau,2,6837,Jack Lemmon,2,3151,Ann-Margret,1,13567,Sophia Loren,1,16757,Daryl Hannah,1,589,Howard Deutch,2,26502
3,31357,Whitney Houston,1,8851,Angela Bassett,1,9780,Loretta Devine,1,18284,Lela Rochon,1,51359,Gregory Hines,2,66804,Forest Whitaker,2,2178
4,11862,Steve Martin,2,67773,Diane Keaton,1,3092,Martin Short,2,519,Kimberly Williams-Paisley,1,70696,George Newbern,2,59222,Charles Shyer,2,56106


In [107]:
# Load the DataFrame
df = pd.read_csv("data/movies_keywords.csv")

# Define a function to simplify and convert types
def simplify_keywords(keyword_str):
    try:
        return ast.literal_eval(keyword_str)
    except (ValueError, SyntaxError):
        return []

# Apply to keywords
keywords = pd.DataFrame()
keywords['id'] = df['id'].astype(str)
keywords['keywords'] = df['keywords'].apply(simplify_keywords)

# Extract the top 5 cast members
for i in range(5):
    keywords[f'keyword_{i+1}_name'] = keywords['keywords'].apply(
        lambda x: x[i]['name'] if len(x) > i else None
    )
    keywords[f'keyword_{i+1}_id'] = keywords['keywords'].apply(
        lambda x: str(x[i]['id']) if len(x) > i else None
    )

# Drop the original 'cast', 'crew', and 'director_info' columns
keywords = keywords.drop(columns=['keywords'])

keywords.head()


,id,keyword_1_name,keyword_1_id,keyword_2_name,keyword_2_id,keyword_3_name,keyword_3_id,keyword_4_name,keyword_4_id,keyword_5_name,keyword_5_id
0,862,jealousy,931,toy,4290,boy,5202,friendship,6054,friends,9713
1,8844,board game,10090,disappearance,10941,based on children's book,15101,new home,33467,recluse,158086
2,15602,fishing,1495,best friend,12392,duringcreditsstinger,179431,old men,208510,None,None
3,31357,based on novel,818,interracial relationship,10131,single mother,14768,divorce,15160,chick flick,33455
4,11862,baby,1009,midlife crisis,1599,confidence,2246,aging,4995,daughter,5600


In [108]:
# Merge the two DataFrames on 'id'
merged_df = pd.merge(metadata, credits, on='id', how='left')
merged_df = pd.merge(merged_df, keywords, on='id', how='left')

display(merged_df.head())

# Save the merged DataFrame to a CSV file
merged_df.to_csv("cleaned_data/movies.csv", index=False)

,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,...,keyword_1_name,keyword_1_id,keyword_2_name,keyword_2_id,keyword_3_name,keyword_3_id,keyword_4_name,keyword_4_id,keyword_5_name,keyword_5_id
0,False,30000000,"Animation, Comedy, Family",862,en,21.946943,Pixar Animation Studios,US,1995-10-30,373554033.0,...,jealousy,931,toy,4290,boy,5202,friendship,6054,friends,9713
1,False,65000000,"Adventure, Fantasy, Family",8844,en,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",US,1995-12-15,262797249.0,...,board game,10090,disappearance,10941,based on children's book,15101,new home,33467,recluse,158086
2,False,0,Comedy,11862,en,8.387519,"Sandollar Productions, Touchstone Pictures",US,1995-02-10,76578911.0,...,baby,1009,midlife crisis,1599,confidence,2246,aging,4995,daughter,5600
3,False,60000000,"Action, Crime, Drama, Thriller",949,en,17.924927,"Regency Enterprises, Forward Pass, Warner Bros.",US,1995-12-15,187436818.0,...,robbery,642,detective,703,bank,974,obsession,1523,chase,3713
4,False,58000000,"Comedy, Romance",11860,en,6.677277,"Paramount Pictures, Scott Rudin Productions, M...","DE, US",1995-12-15,0.0,...,paris,90,brother brother relationship,380,chauffeur,2072,long island,9398,fusion,9492


In [109]:
# Load your credits CSV
df = pd.read_csv("data/credits.csv")

# Define a function to extract name, id, and gender from cast JSON
def extract_cast_info(cast_json_str):
    try:
        cast_list = ast.literal_eval(cast_json_str)
        return [{'name': member['name'], 'id': str(member['id']), 'gender': str(member['gender'])} for member in cast_list]
    except:
        return []
    
# Define a function to extract director info from crew JSON
def extract_director_info(crew_json_str):
    try:
        crew_list = ast.literal_eval(crew_json_str)
        return [{'name': member['name'], 'id': str(member['id']), 'gender': str(member['gender'])}
                for member in crew_list if member.get('job') == 'Director']
    except:
        return []

# Apply the function to the 'cast' column
df['simplified_cast'] = df['cast'].apply(extract_cast_info)
# Apply the function to the 'crew' column
df['simplified_directors'] = df['crew'].apply(extract_director_info)

# explode to have one row per cast member (with name, id, gender)
exploded_actors_df = df.explode('simplified_cast')
exploded_actors_df = pd.json_normalize(exploded_actors_df['simplified_cast'])

# explode to have one row per director (with name, id, gender)
exploded_directors_df = df.explode('simplified_directors')
exploded_directors_df = pd.json_normalize(exploded_directors_df['simplified_directors'])

# Show result
display(exploded_actors_df.head())
display(exploded_directors_df.head())

# Save the exploded DataFrame to a CSV file
exploded_actors_df.to_csv("cleaned_data/actors.csv", index=False)
exploded_directors_df.to_csv("cleaned_data/directors.csv", index=False)


,name,id,gender
0,Tom Hanks,31,2
1,Tim Allen,12898,2
2,Don Rickles,7167,2
3,Jim Varney,12899,2
4,Wallace Shawn,12900,2


,name,id,gender
0,John Lasseter,7879,2
1,Joe Johnston,4945,2
2,Howard Deutch,26502,2
3,Forest Whitaker,2178,2
4,Charles Shyer,56106,2


In [110]:
# Load your credits CSV
df = pd.read_csv("data/movies_keywords.csv")

# Define a function to extract name, id, and gender from cast JSON
def extract_keyword_info(keyword_json_str):
    try:
        keyword_list = ast.literal_eval(keyword_json_str)
        return [{'name': keyword['name'], 'id': str(keyword['id'])} for keyword in keyword_list]
    except:
        return []

# Apply the function to the 'keywords' column
df['simplified_keywords'] = df['keywords'].apply(extract_keyword_info)

# explode to have one row per cast member (with name, id, gender)
exploded_keywords_df = df.explode('simplified_keywords')
exploded_keywords_df = pd.json_normalize(exploded_keywords_df['simplified_keywords'])

# Show result
display(exploded_keywords_df.head())

# Save the exploded DataFrame to a CSV file
exploded_keywords_df.to_csv("cleaned_data/keywords.csv", index=False)


,name,id
0,jealousy,931
1,toy,4290
2,boy,5202
3,friendship,6054
4,friends,9713


The above code downloads, cleans, and merges multiple datasets from the movies database. It starts with the metadata dataset and breaks up the columns that contain objects. It then merges the credits dataset with the metadata dataset, keeping the name, gender, and id of the top 5 cast members as well as the director. It then merges the keywords dataset, keeping the top 5 keywords along with their ids. The final dataset is saved as a CSV file for further analysis. Finally, it explodes the cast, crew, and keyword info into their own datasets so that we have access to the mappings of the ids to the names.